In [1]:
%%capture
!apt-get update
!apt-get install -y pandoc
!pip install langchain langchain-google-genai langchain-community faiss-cpu langchain-huggingface pypdf openai langchain_openai pypandoc Spire.Doc
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install torch torchvision torchaudio xformers --index-url https://download.pytorch.org/whl/cu121

In [2]:
import os
import uuid
import base64
from IPython.display import Markdown, Image
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.schema.messages import HumanMessage, SystemMessage
from langchain.schema.document import Document
from langchain.vectorstores import FAISS
from langchain.retrievers.multi_vector import MultiVectorRetriever
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from operator import itemgetter
import os
from pprint import pprint
from typing import List
from langchain_core.documents import Document
from typing_extensions import TypedDict
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.utilities import DuckDuckGoSearchAPIWrapper, BraveSearchWrapper
from langchain_community.tools import BraveSearch
from langchain_core.example_selectors import SemanticSimilarityExampleSelector
from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)
from langchain.chains import ConversationChain
from langchain.memory import ConversationSummaryBufferMemory
from langchain.prompts import PromptTemplate,ChatPromptTemplate
from langchain.document_loaders import PyPDFLoader

import warnings
warnings.filterwarnings('ignore')
from google.colab import files

from langchain.load import dumps, loads
from langchain_openai import ChatOpenAI

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Run this only if you want to generate new FAISS index for another stock documents

In [3]:
def load_pdf_files(folder_path):
    """
    Loads PDF files from a given folder, processes them using PyPDFLoader,
    and returns the extracted data.

    Args:

        folder_path (str): Path to the folder containing PDF files.

    Returns:
        list: Extracted data from all successfully processed PDF files.
    """
    pdf_files = [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith('.pdf')]
    extracted_data = []

    for pdf_file in pdf_files:
        try:
            loader = PyPDFLoader(pdf_file)
            data = loader.load()
            extracted_data.extend(data)
            print(f"Successfully loaded: {pdf_file}")
        except Exception as e:
            print(f"Error loading {pdf_file}: {e}")

    return extracted_data

In [4]:
import os
import shutil

def zip_folder(folder_path, output_filename=None):
    """
    Zips the contents of a folder and saves it as a zip file.

    Args:
        folder_path (str): Path to the folder to be zipped.
        output_filename (str): Optional; Output path for the zip file.
                               If not provided, it will use the folder name.
    """
    # Extract folder name from the folder path
    folder_name = os.path.basename(folder_path.rstrip('/\\'))

    # Derive output filename if not provided
    if output_filename is None:
        output_filename = os.path.join(os.path.dirname(folder_path), folder_name)

    # Create the zip file
    shutil.make_archive(output_filename, 'zip', folder_path)
    print(f"Zipped folder '{folder_path}' into '{output_filename}.zip'")

In [4]:
stock_name = "JKTYRE"

In [5]:
# folder_path = f"/content/drive/MyDrive/stock_research_report_data/{stock_name}/stock_report_documents"
folder_path="/kaggle/input/final-pdf-files"
stock_report_docs = []
stock_report_docs = load_pdf_files(folder_path)
chunk_size = 4096
text_splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_size*0.05)
complete_stock_report_docs = text_splitter.split_documents(stock_report_docs)

Successfully loaded: /kaggle/input/final-pdf-files/Transcript_JKTYRE.pdf
Successfully loaded: /kaggle/input/final-pdf-files/ppt_JKTYRE.pdf
Successfully loaded: /kaggle/input/final-pdf-files/icra_rating_nov_2024.pdf
Successfully loaded: /kaggle/input/final-pdf-files/FINANCIAL_DATA_FOR_RESEARCH_REPORT - JKTYRE.pdf
Successfully loaded: /kaggle/input/final-pdf-files/annual_report_JKTYRE.pdf


In [6]:
from langchain_community.embeddings import HuggingFaceBgeEmbeddings

embd_model_name = "Alibaba-NLP/gte-large-en-v1.5" #"dunzhang/stella_en_400M_v5"  # Embedding dimension is 1024  # "dunzhang/stella_en_400M_v5""dunzhang/stella_en_1.5B_v5"
model_kwargs = {
    'device': 'cuda',
    'trust_remote_code': True
}
encode_kwargs = {'normalize_embeddings': False}

embeddings = HuggingFaceBgeEmbeddings(
    model_name=embd_model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs,
    # query_instruction="search_query:",
    # embed_instruction="search_document:"
)


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

configuration.py:   0%|          | 0.00/7.13k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

## Run if you want generate the vector embeddings for another stock

In [7]:
import os
stock_folder_path = f"/kaggle/working/{stock_name}"
if not os.path.exists(stock_folder_path):
    os.makedirs(stock_folder_path)
    print(f"Folder '{stock_name}' created at {stock_folder_path}")
else:
    print(f"Folder '{stock_name}' already exists.")

stock_markdown_folder_path = f"/kaggle/working/{stock_name}/markdown"
if not os.path.exists(stock_markdown_folder_path):
    os.makedirs(stock_markdown_folder_path)
    print(f"Folder 'markdown' created at {stock_markdown_folder_path}")
else:
    print(f"Folder 'markdown' already exists.")

Folder 'JKTYRE' created at /kaggle/working/JKTYRE
Folder 'markdown' created at /kaggle/working/JKTYRE/markdown


In [8]:
vectorstore = FAISS.from_documents(documents=complete_stock_report_docs, embedding=embeddings)
vectorstore_path = f"/kaggle/working/{stock_name}/{stock_name}_stock_report_faiss_index_{embd_model_name.split('/')[1]}_{chunk_size}"
vectorstore.save_local(vectorstore_path)
# zip_folder(vectorstore_path)
# files.download(f"{vectorstore_path}.zip")

#############################################################################################

## Loading pre-stored FAISS index of the stock

In [9]:
db_stock_report = FAISS.load_local(vectorstore_path, embeddings, allow_dangerous_deserialization=True)
retriever_stock_report = db_stock_report.as_retriever(search_type="similarity",search_kwargs={"k": 20})

# Multi Query: Different Perspectives


In [11]:
def get_unique_union(documents: list[list]):
    """ Unique union of retrieved docs """
    # Flatten list of lists, and convert each Document to string
    flattened_docs = [dumps(doc) for sublist in documents for doc in sublist]
    # Get unique documents
    unique_docs = list(set(flattened_docs))
    # Return
    return [loads(doc) for doc in unique_docs]

def reciprocal_rank_fusion(results: list[list], k=60):
    """ Reciprocal_rank_fusion that takes multiple lists of ranked documents
        and an optional parameter k used in the RRF formula """

    # Initialize a dictionary to hold fused scores for each unique document
    fused_scores = {}

    # Iterate through each list of ranked documents
    for docs in results:
        # Iterate through each document in the list, with its rank (position in the list)
        for rank, doc in enumerate(docs):
            # Convert the document to a string format to use as a key (assumes documents can be serialized to JSON)
            doc_str = dumps(doc)
            # If the document is not yet in the fused_scores dictionary, add it with an initial score of 0
            if doc_str not in fused_scores:
                fused_scores[doc_str] = 0
            # Retrieve the current score of the document, if any
            previous_score = fused_scores[doc_str]
            # Update the score of the document using the RRF formula: 1 / (rank + k)
            fused_scores[doc_str] += 1 / (rank + k)

    # Sort the documents based on their fused scores in descending order to get the final reranked results
    reranked_results = [
        (loads(doc), score)
        for doc, score in sorted(fused_scores.items(), key=lambda x: x[1], reverse=True)
    ]

    # Return the reranked results as a list of tuples, each containing the document and its fused score
    return reranked_results

In [10]:
multi_query_template = """You are an AI language model assistant. Your task is to generate Five
different versions of the given user question to retrieve relevant documents from a vector
database. By generating multiple perspectives on the user question, your goal is to help
the user overcome some of the limitations of the distance-based similarity search.
Provide these alternative questions separated by newlines.

Original question: {question}
"""

In [11]:
template_content_review = """You are an Expert Content Reviewer in the Financial Domain.
Your task is to refine the provided content by removing unwanted disclaimers, mentions of information gaps, and
suggestions/recommendations based on these guidelines:

Content to Remove:
Disclaimers: Eliminate phrases like "This is not official advice" or similar.
Information Gaps: Remove references to missing data or incomplete analysis, such as "Further details are needed."
Suggestions/Recommendations: Exclude sentences suggesting future actions, like "Consider exploring additional areas."
Keywords: Delete sentences with words like "further," "require," or "consider" that imply disclaimers or gaps.

Formatting Requirements:
Maintain Markdown format.
Ensure proper use of headings (#, ##, ###) and bullet points (•, *).
For tables, bold all column names and the first column values.

Content Quality:
Preserve clarity, professionalism, and relevance.
Ensure content is concise and compliant with the specified criteria.

Review Content:
{content}"""

In [12]:
template_content_review_new = """You are an Expert Content Reviewer in the Financial Domain.
Your task is to refine the provided content by removing unwanted disclaimers, mentions of information gaps, and
suggestions/recommendations based on these guidelines:

Content to Remove:
Disclaimers: Eliminate phrases like "This is not official advice" or similar.
Information Gaps: Remove references to missing data or incomplete analysis, such as "Further details are needed."
Suggestions/Recommendations: Exclude sentences suggesting future actions, like "Consider exploring additional areas."
Keywords: Delete sentences with words like "further," "require," or "consider" that imply disclaimers or gaps.

Formatting Requirements:
Maintain Markdown format.
Ensure proper use of headings (#, ##, ###) and bullet points (•, *).
For tables, bold all column names and the first column values.

Content Quality:
Preserve clarity, professionalism, and relevance.
Ensure content is concise and compliant with the specified criteria.

Review Content:
{content}"""

In [13]:
template_stock_report = """

Here’s an updated version of the instructions with the additional guideline included:

You are an LLM-powered Research Analyst tasked with creating objective, data-driven reports on publicly traded companies
using only the provided context (e.g., annual reports, financial data).

Objective
Deliver a comprehensive, structured analysis strictly based on the given data.

Guidelines
Context-Only: Use only the provided evidence. Avoid speculation or external inputs.
Clarity and Structure: Organise the report with sub-headings, bullet points, and concise paragraphs. Verify all numerical data for accuracy
and coherence.
Professional Tone: Provide balanced, impartial insights in UK English, highlighting risks and opportunities.
Interpretation Over Repetition: Do not include the provided values or numbers verbatim. Instead, integrate them into the analysis with
insights and reasoning, explaining their reasons and implications.

Deliverables
Factual, data-backed insights linking trends to impacts.
Clear, structured, and impartial reporting with no unnecessary duplication of numerical data.
While doing the Analysis of Financial Data, must include the numerical data & changes in data over the period of time.

Goal: Generate a comprehensive and reliable research report using only the given documentary evidence.

Question: {question}
Context: {context}
"""

In [14]:
from huggingface_hub import login
login(token = "")

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [15]:
from langchain.llms import HuggingFacePipeline

from transformers import AutoTokenizer

import transformers

import torch

model =  "google/gemma-2-2b-it"#"unsloth/Llama-3.2-3B" #"unsloth/mistral-7b-v0.3"

tokenizer=AutoTokenizer.from_pretrained(model)

pipeline = transformers.pipeline(

    "text-generation",

    model=model,

    tokenizer=tokenizer,

    torch_dtype=torch.bfloat16,

    trust_remote_code=True,

    # max_length=4096,

    #return_full_text=False,

    do_sample=True,

    top_k=10,

    num_return_sequences=1,

    eos_token_id=tokenizer.eos_token_id,

    max_new_tokens=2048,

    device=0

    )

llm=HuggingFacePipeline(pipeline=pipeline, model_kwargs={'temperature':0,"device":"cuda"})

tokenizer_config.json:   0%|          | 0.00/47.0k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/24.2k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/241M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/187 [00:00<?, ?B/s]

In [17]:
# llm_generation = ""

llm_review = ChatGoogleGenerativeAI(model="gemini-1.5-pro", temperature=0, convert_system_message_to_human=True, google_api_key="")



prompt_multi_query = ChatPromptTemplate.from_template(multi_query_template)
prompt_content_review = ChatPromptTemplate.from_template(template_content_review)
prompt_stock_report = ChatPromptTemplate.from_template(template_stock_report)

# def get_final_response(question, llm_review = llm_review, llm_generation = llm_generation):

#   generate_queries = prompt_multi_query  | llm_review  | StrOutputParser() | (lambda x: [line for line in x.split("\n") if line.strip()] )
#   retrieval_chain_rag_fusion_for_stock_report = generate_decomposed_queries | retriever_stock_report.map() |  get_unique_union  | reciprocal_rank_fusion
#   final_rag_chain_stock_report = (
#       {"context": retrieval_chain_rag_fusion_for_stock_report,
#       "question": itemgetter("question")}
#       | prompt_stock_report
#       | llm_generation
#       | StrOutputParser()
#   )
#   response = final_rag_chain_stock_report.invoke({"question": question})
#   content_review_chain = (prompt_content_review | llm_review | StrOutputParser())

#   return content_review_chain.invoke({"content": response})


In [18]:
from langchain_core.prompts import ChatPromptTemplate

# prompt =  """You are a content reviewer and can extract the answer which is in
#             in the provided content and do not generate any sentence from your own like
#             'Here is the extracted company overview in a structured format'
#             just the content. 
#             You are tasked with to extract that answer content only as ouput without any suffix or prefix.
#            Here is the content for extraction:{content}"""
        
# prompt_ans_generation = ChatPromptTemplate.from_template(prompt)



# Query generation chain

generate_ans = (

    prompt_content_review

    | llm_review

    | StrOutputParser()

)


In [19]:
from langchain_core.output_parsers import StrOutputParser

from langchain.retrievers import BM25Retriever, EnsembleRetriever

ensemble_retriever = EnsembleRetriever(retrievers=[retriever_stock_report])

from langchain.chains import RetrievalQA

# ensemble_retriever_annual = EnsembleRetriever(retrievers=[retriever_annual])

In [20]:
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
hybrid_chain = RetrievalQA(
    combine_documents_chain=StuffDocumentsChain(
        verbose=False,
        llm_chain=LLMChain(
            verbose=False,
            llm=HuggingFacePipeline(model_kwargs={'temperature': 0, 'device': 'cuda'}),
            prompt=PromptTemplate(
                input_variables=['context', 'question'],
                template=template_stock_report,
            )
        ),document_variable_name="context"
    ),retriever=ensemble_retriever,
).from_chain_type(
    llm=llm, chain_type="stuff", retriever=ensemble_retriever, 
)

In [21]:
question_company_overview = """ Extract a comprehensive Company/Stock Overview in structured format, do not include any type of financial data.
You may include following details about Company:

Header : '1. Company Overview’
(Content for all sub-headers for ‘1. Company Overview’ can be extracted from annual / quarterly reports, earnings call transcripts,
investor presentations)

1.1 Company Introduction :
(Include the following and don't write it like a definition, write it in such a way that it pulls in the reader and keep it within 800 words):
Founding year, founders, and origin
Management Team (not more than 2 to 3 personals,include names, roles of top management - CEO, CFO, etc) in Bullet format.
Key historical milestones like expansions, acquisitions, product launches

Overview of current operations, production capacities, and locations

1.2. Business Model : (Make it structured but short & concise and include the following and keep it within 1500 words):
Major revenue streams such as  product sales, subscription, licensing etc
Value proposition - cost leadership, premium branding
Revenue-generating structure like B2B, B2C, etc """

question_revenue_segments = """ Extract a comprehensive Revenue Segments Overview in structured format.

Sub-header : 1.3. Revenue Segments (Include the following):

Following points are sub-sub-headers :

1.3.1. Product & Service breakdown:
(Keep it within 700 words without missing out on details and without writing vague things, if some detail is not available then skip it)
Contribution and performance
List of major product lines and/or services
Revenue contribution by product/service in Table format or any structured way if possible. If not then in descriptive format.
Historical growth rates for each segment

1.3.2. Geographical breakdown:
(Keep it within 300 words without missing out on details and without writing vague things, if some detail is not available then skip it)
Domestic vs. international revenue breakdown with analysis (if company not present internationally, then domestic state wise breakdown)
Growth trends in major regions
Market share in major regions

Instructions:
Add header and sub-headers as only mentioned

"""

question_competitive_positioning = """ Extract the Comprehensive Structured Report on Competitive Positioning :


Sub-header : 1.4.  Competitive Positioning

sub-sub-headers:
                  1.4.1. Strengths (example - mkt leader, innovation, cost advantages etc) in atleast 300 words.
                  1.4.2. Weaknesses (example - concentrated revenue base, debt levels etc) in atleast 300 words.
                  1.4.3. Opportunities (example - mkt expansion, product diversification etc) in atleast 300 words.
                  1.4.4. Threats (example - competitor innovation, cost pressures like raw material cost, etc) in atleast 300 words.
                  1.4.5. Bargaining power of suppliers and buyers in atleast 300 words.
                  1.4.6. Threat of substitutes and new entrants in atleast 300 words.
                  1.4.7. Competitive intensity within the industry in atleast 300 words.
                  1.4.8. Impact of government policies in atleast 300 words.
                  1.4.9. Key Differentiators and Moats (example - specific competitive moats like capex, patents, economies of scale) in atleast 300 words.
Instructions:
Add header and sub-headers as only mentioned

"""

question_profit_loss = """Analyze the Only Profit & Loss statement, Balance Sheet of the company,
focusing on the following aspects:

Header : ‘2. Financial Analysis’

Sub-headers:
      2.1. Analysis of Financial Results :

Sub-sub-header: 2.1.1.  Analysis of Profit & Loss Statement
        Note: Do concise analysis with justification/reasons for changes in the values across the period. Do not add values or changes in values as it is

        Last 4 quarter's Analysis in Structured format including all available values
        Analysis of Historical Profit & Loss Statement in concise but as structured format as possible including all available values.

Keep this section of 'Analysis of Financial Results' within 300 words without missing out on details and without writing vague things,
if some detail is not available then skip it.

Instruction :
Use the Financial Data mentioned in Tabular form, but do not add the Table or in any format as it is, in any case, in the Response.
Add header and sub-headers as only mentioned
"""

question_balance_sheet = """Analyze the Only Balance Sheet of the company,
focusing on the following aspects:

Sub-sub-header: 2.1.2. Balance Sheet Analysis
Do concise analysis with justification/reasons for changes in the values across the period. Do not add values or changes in values as it is
    Last 4 quarter's Analysis in Structured format including all available values
    Analysis of Historical Balance Sheet Statement in concise but as structured format as possible including all available values.

Keep this section of 'Analysis of Financial Results' within 300 words without missing out on details and without writing vague things,
if some detail is not available then skip it.

Instruction :
Use the Financial Data mentioned in Tabular form, but do not add the Table or in any format as it is, in any case, in the Response.
Add header and sub-headers as only mentioned
"""


question_cash_flow_ratios = """Extract the Cash Flow Statement & Financial Ratios only & analyze them in most concise way possible.

Sub-sub-header : 2.1.3. Cash Flow Statement Analysis

Do concise analysis with justification/reasons for changes in the values across the period. Do not add values or changes in values as it is
      Last 4 quarter's Analysis in Structured format including all available values
      Analysis of Historical Cash Flow Statement in concise but as structured format as possible including all available values.

Sub-header: 2.2. Financial Ratio Analysis :
                                1. Provide the cummulative analysis(include main values in the analysis) of annual Historical Ratio Data,
                                   not ratio-wise analyis.Do not make it structured but make it detailed-one, but include the values and possible reasons
                                   for changes in values.
                                2. Keep the response for this section upto 500 words

Note : Refrain from writing filler sentences which don't have any meaning which sound like its just being written to fill space or reach
      the word count.

Instruction :
Use the Financial Data mentioned in Tabular form, but do not add the Table or in any format as it is, in any case, in the Response.
Add header and sub-headers as only mentioned

"""

question_segment_performance_analysis = """ Extract the Comprehensive Structured Report on Segment Performance Analysis

Sub-header: 2.3. Segment Performance Analysis
(Keep it within 1000 words)

sub-sub-header :
              Key Segments: Identification of the company’s major segments contributing to overall revenue, such as product categories or service offerings.
              Major Geographical Presence: Revenue Breakdown by Geography & Segment (include the revenue generated in the major segments and
              the major locations and if historical record available then use that also)
              Growth Trends: Year-over-year performance, with a focus on high-growth segments.
              Market Share and Competitive Position of major contributing segments
              Market Position: The company’s position in key market segments compared to competitors.
              Competitive Advantages: Unique offerings or strengths that differentiate the company in each segment
Instructions:
Add header and sub-headers as only mentioned
"""

question_industry_analysis = """Overview of the Indian Industry and the Company’s Role

Header : 3. Industry Analysis
(Content for all sub-headers for ‘3. Industry Analysis’ can be extracted from earnings call transcripts, investor presentations, news)
(Make the content under each sub-header structured one, do not add information under the sub-headers as a single paragraph.)

Sub-headers :
              3.1. Industry Dynamics
              (Keep it within 800 words)
              Industry growth rates - historical and projected
              Emerging trends (Example - digitization, sustainability)
              Key regulations or policies impacting the industry
              3.2. Key Drivers
              (Keep it within 800 words)
              Macro trends (examples - urbanization, rising incomes, commercial growth, manufacturing growth etc)
              Innovations driving demand (examples - AI, renewables etc)
              Sector-specific growth drivers (examples - food security for agriculture, EV adoption for auto etc)

Instructions:
Add header and sub-headers as only mentioned

"""

question_peer_analysis = """ Extract the Peer Analysis in most comprehensive & structured way possible for the mentioned period.

Sub-Headers :
3.3. Peer Analysis
(Keep it within 800 words)

(Include all parameters included in the table for all peers in the analysis as structured as possible and DON'T INCLUDE THE PEER
COMPARISON IN TABLE FORMAT..JUST DO THE ANALYSIS. Also don't do the analysis stock-wise, do it parameter wise.)
Competitive landscape (examples - list of key competitors)

Performance Comparison: Compare the Stock's and its Peer's absolute returns with the respective index across various time horizons.
Market share (examples - comparative market share of peers)
Financial Ratio Analysis comparing peers (Include only available ratios in the response, avoid mentioning ratios which are not having any value
Relative strengths (examples - technological or strategic advantage / disadvantage, long term supplier contracts, etc)
in the context or in the sector which the company belongs in)

Instructions:
Add header and sub-headers as only mentioned
Use the Financial Data mentioned in Tabular form, but do not add the Table or in any format as it is, in any case, in the Response.
"""

question_expansion_future_plan = """ Extract details about the company's Expansion and Future Plans in the
most comprehensive & structured way possible for the mentioned period.

Header : ‘4. Expansions and Long Term Objectives’
          This section is to outline the strategic expansion and growth plans of the company, focusing on production capacity, new product development,
          market expansion, R&D and investment strategies to strengthen its industry position.

Sub-headers:
          4.1. Capacity Expansion:
          (Keep it within 600 words)
          New Facilities: Examine the launch of new production units or facilities, including investment amounts, production capacities, and their
          strategic significance.
          Planned Expansions: Discuss upcoming capacity expansion plans, expected dates, capital expenditure, and their role in meeting growing demand.
          4.2. New Product Development:
          (Keep it within 600 words)
          Product Launches: Detail upcoming product introductions and their expected impact on the market.
          R&D Efforts: Highlight the company's research and development initiatives, including new innovations and the focus on high-value products.
          4.3. Market Expansion:
          (Keep it within 600 words)
          Geographic Reach: Describe efforts to expand into new markets and the strategy behind global distribution.
          Diversification Strategy: Analyze the company's approach to reducing reliance on any single market, positioning itself as a competitive
          alternative in global supply chains.
          4.4. Investments and Funding:
          (Keep it within 500 words)
          Funding Approach: Outline how the company is financing its expansion plans.
          Financial Discipline: Discuss the company’s ability to fund major projects while maintaining financial stability.
          4.5. Key Takeaways and Future Outlook:
          (Keep it within 800 words)
          Summarize the company’s expansion strategy, focus on innovation, and strong financials and do the analysis based on material available
          information.

Instructions:
Add header and sub-headers as only mentioned
"""

question_esg_initiatives = """

Header : ‘5. ESG Initiatives’

sub-headers :
          5.1. Environmental Strategies
          (Keep it within 600 words)
          Commitment to reducing environmental impact through energy efficiency, emissions reduction, and sustainable resource management.
          Adoption of green technologies and renewable energy sources to support long-term sustainability.
          Implementation of waste reduction and recycling initiatives across operations.
          5.2. Social Responsibility
          (Keep it within 600 words)
          Focus on enhancing community engagement through social programs, charitable contributions, and volunteer efforts.
          Promotion of workplace diversity, equity, and inclusion across all levels of the organization.
          Ensuring fair labor practices and ethical sourcing throughout the supply chain.
          5.2. Governance Standards
          (Keep it within 600 words)
          Strong corporate governance policies ensure transparency, accountability, and ethical decision-making.
          Regular independent audits and risk management processes to uphold investor trust.
          Development of robust mechanisms for shareholder rights and stakeholder communication.
          5.3. ESG Commitments
          (Keep it within 500 words)
          Setting clear, measurable ESG targets with regular progress reviews and public disclosures.
          Investment in innovative practices and technologies to achieve long-term ESG goals.
          Continuous alignment with global ESG standards, regulations, and frameworks for improved performance.

Instructions:
Add header and sub-headers as only mentioned

"""


In [22]:
# questions_dict = {
#     question_company_overview: "annual",
#     question_revenue_segments: "annual",
#     question_competitive_positioning: "annual",
#     question_profit_loss: "financial",
#     question_balance_sheet: "financial",
#     question_cash_flow_ratios: "financial",
#     question_segment_performance_analysis: "annual",
#     question_industry_analysis: "annual",
#     question_peer_analysis: "financial",
#     question_expansion_future_plan:"annual",
#     question_esg_initiatives : "annual"

# }

question_list = [
    question_company_overview,
    question_revenue_segments,
    question_competitive_positioning,
    question_profit_loss,
    question_balance_sheet,
    question_cash_flow_ratios,
    question_segment_performance_analysis,
    question_industry_analysis,
    question_peer_analysis,
    question_expansion_future_plan,
    question_esg_initiatives
]


In [23]:
import threading
import re

final_stock_research_report = f"# Comprehensive Research Report on {stock_name} "

display(Markdown(final_stock_research_report))


for question in question_list[4:7]:

    #if chain_type == "annual":

    response = hybrid_chain.invoke({"query":question})

    # elif chain_type == "financial":

    #     response = hybrid_chain_financial.invoke({"query": question})

    ans = generate_ans.invoke({"content": response["result"]})
                              
    # reviewed_response = content_review_chain.invoke({"content": ans})

    # pattern = r".*?Review the following content:\n"
    # cleaned_content = remove_intro(reviewed_response)

    display(Markdown(ans))

    final_stock_research_report += "\n" + ans + "\n"

# Comprehensive Research Report on JKTYRE 

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.00 GiB. GPU 0 has a total capacity of 14.74 GiB of which 1.78 GiB is free. Process 2241 has 12.96 GiB memory in use. Of the allocated memory 12.26 GiB is allocated by PyTorch, and 591.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
final_stock_research_report = f"# Comprehensive Research Report on {stock_name} "
display(Markdown(final_stock_research_report))

final_list = []
for question in question_list:
  reviewed_response = get_final_response(question, llm_review = llm_review, llm_generation = llm)
  display(Markdown(reviewed_response))
  final_stock_research_report += "\n" + reviewed_response + "\n"

In [ ]:
Markdown(final_stock_research_report)

In [ ]:
with open(f"/content/{stock_name}/markdown/{stock_name}_Research_Report.md", "w") as file:
    file.write(final_stock_research_report)

In [ ]:
import pypandoc

def convert_md_to(format, input_file, output_folder):

    if not os.path.exists(input_file):
        raise FileNotFoundError(f"The file {input_file} does not exist.")

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    base_name = os.path.basename(input_file).replace('.md', f'.{format}')
    output_file = os.path.join(output_folder, base_name)

    try:
        pypandoc.convert_file(input_file, format, outputfile=output_file)
        print(f"Converted {input_file} to {output_file}")
        return output_file
    except Exception as e:
        raise RuntimeError(f"Conversion failed: {e}")

def batch_convert_md(folder, output_folder, formats):

    md_files = [f for f in os.listdir(folder) if f.endswith('.md')]
    for md_file in md_files:
        input_path = os.path.join(folder, md_file)
        for format in formats:
            convert_md_to(format, input_path, output_folder)

# Configuration
input_folder = stock_markdown_folder_path  # Folder containing .md files
output_folder = stock_markdown_folder_path
formats_to_convert = ['docx', 'html']  # Desired output formats

# Execute the conversion
batch_convert_md(input_folder, output_folder, formats_to_convert)